In [ ]:
!pip3 install pycuda

     |████████████████████████████████| 1.6MB 2.5MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4537409 sha256=10d9bf315ebdf1d9e6266b11aeb199fd570eef71501fbd116d7e14e940a403c5
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2020.4-py2.py3-none-any.whl size=67175 sha256=b9e49bce99e9d0fd01766ee5e0ee7f385be8f2877833a7eff5f8e1e9628e96ce
  Stored in directory: /root/.cache/pip/wheels/5e/20/0b/fac51840734b2587ecc239a62522b164c374e929e2c9be66c5
Successfully built pycuda pytools


In [ ]:
import os

In [ ]:
os.mkdir("cuda")

In [ ]:
import math
import numpy as np
import time
import pycuda.gpuarray as gpuarray
import pycuda.driver as drv
from pycuda.compiler import SourceModule

In [ ]:
import pycuda.autoinit

In [ ]:
cuda_file_path = os.path.abspath("./cuda")

In [ ]:
module = SourceModule("""
#include "kernel_functions_for_math_1d.cu"
""", include_dirs=[cuda_file_path])

In [ ]:
plus_one_kernel = module.get_function("plus_one_kernel")

In [ ]:
num_components = np.int32(1e6)
x = np.arange(num_components, dtype=np.int32)

In [ ]:
x_gpu = gpuarray.to_gpu(x)
y_gpu = gpuarray.zeros(num_components, dtype=np.int32)

In [ ]:
threads_per_block = (256, 1, 1)
blocks_per_grid = (math.ceil(num_components / threads_per_block[0]), 1, 1)

In [ ]:
plus_one_kernel(num_components, y_gpu, x_gpu, block=threads_per_block, grid=blocks_per_grid)

In [ ]:
y_gpu.get()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int32)

In [ ]:
time_start_cpu = time.time()
x = x + 1
time_end_cpu = time.time()

In [ ]:
print("CPU calculation {0} [msec]".format(1000 * (time_end_cpu - time_start_cpu)))

CPU calculation 3.698587417602539 [msec]


In [ ]:
time_start_gpu = drv.Event()
time_end_gpu = drv.Event()

In [ ]:
time_start_gpu.record()
plus_one_kernel(num_components, y_gpu, x_gpu, block=threads_per_block, grid=blocks_per_grid)
time_end_gpu.record()
time_end_gpu.synchronize()

In [ ]:
print("GPU calulation {0} [msec]".format(time_start_gpu.time_till(time_end_gpu)))

GPU calulation 0.2502079904079437 [msec]


In [ ]:
time_start_gpu.record()
y_gpu.get()
time_end_gpu.record()
time_end_gpu.synchronize()

In [ ]:
print("GPU calulation {0} [msec]".format(time_start_gpu.time_till(time_end_gpu)))

GPU calulation 2.6345279216766357 [msec]
